## LIBS

In [62]:
import os
from time import sleep
import pandas as pd
import csv
from datetime import datetime
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from sqlalchemy import create_engine

In [63]:
Download_Dir = {
    'MailChimp': '/Users/karim/Downloads/Mailchimp_downloads/'
}


col_renames = {
  'Unsubscribed':{
        'Email Address':'EMAIL',
        'First Name':'FIRST_NAME', 
        'Last Name':'LAST_NAME',
        'Address':'ADDRESS', 
        'Phone Number':'PHONE',
        'Company':'COMPANY',
        'Full Name':'FUll_NAME',
        'Company Size':'COMPANY_SIZE',
        'Member Rating':'MEMBER_RATING',
        'reason':'REASON',
        'description':'DESCRIPTION'},

    'Didnt_open':{
        'Email Address':'EMAIL',
        'First Name':'FIRST_NAME', 
        'Last Name':'LAST_NAME',
        'Address':'ADDRESS', 
        'Phone Number':'PHONE',
        'Company':'COMPANY',
        'Full Name':'FUll_NAME',
        'Company Size':'COMPANY_SIZE',
        'Member Rating':'MEMBER_RATING'},

    'Bounced':{
        'Email Address':'EMAIL',
        'First Name':'FIRST_NAME', 
        'Last Name':'LAST_NAME',
        'Address':'ADDRESS', 
        'Phone Number':'PHONE',
        'Company':'COMPANY',
        'Full Name':'FUll_NAME',
        'Company Size':'COMPANY_SIZE',
        'Member Rating':'MEMBER_RATING',
        'Bounce Type':'BOUNCE_TYPE'},
    
    'Clicked':{
        'Email Address':'EMAIL',
        'First Name':'FIRST_NAME', 
        'Last Name':'LAST_NAME',
        'Address':'ADDRESS', 
        'Phone Number':'PHONE',
        'Company':'COMPANY',
        'Full Name':'FUll_NAME',
        'Company Size':'COMPANY_SIZE',
        'Member Rating':'MEMBER_RATING',
        'URL':'URL',
        'Clicks':'CLICKS'},
    
    'Opened':{
        'Email Address':'EMAIL',
        'First Name':'FIRST_NAME', 
        'Last Name':'LAST_NAME',
        'Address':'ADDRESS', 
        'Phone Number':'PHONE',
        'Company':'COMPANY',
        'Full Name':'FUll_NAME',
        'Company Size':'COMPANY_SIZE',
        'Member Rating':'MEMBER_RATING',
        'Opens':'OPENS'},
    
    'Sent_to':{
        'Email Address':'EMAIL',
        'First Name':'FIRST_NAME', 
        'Last Name':'LAST_NAME',
        'Address':'ADDRESS', 
        'Phone Number':'PHONE',
        'Company':'COMPANY',
        'Full Name':'FUll_NAME',
        'Company Size':'COMPANY_SIZE',
        'Member Rating':'MEMBER_RATING'},
}  


db_cols = {
    'Unsubscribed':['EMAIL','FIRST_NAME','LAST_NAME','ADDRESS', 'PHONE','COMPANY','FUll_NAME','COMPANY_SIZE','MEMBER_RATING','REASON','DESCRIPTION'],

    'Didnt_open':['EMAIL','FIRST_NAME','LAST_NAME','ADDRESS','PHONE','COMPANY','FUll_NAME','COMPANY_SIZE','MEMBER_RATING'],

    'Bounced':['EMAIL','FIRST_NAME', 'LAST_NAME','ADDRESS','PHONE','COMPANY','FUll_NAME','COMPANY_SIZE','MEMBER_RATING','BOUNCE_TYPE'],
    
    'Clicked':['EMAIL','FIRST_NAME', 'LAST_NAME','ADDRESS', 'PHONE','COMPANY','FUll_NAME','COMPANY_SIZE','MEMBER_RATING','URL','CLICKS'],
    
    'Opened':['EMAIL','FIRST_NAME', 'LAST_NAME','ADDRESS', 'PHONE','COMPANY','FUll_NAME','COMPANY_SIZE','MEMBER_RATING','OPENS'],
    
    'Sent_to':['EMAIL','FIRST_NAME', 'LAST_NAME','ADDRESS', 'PHONE','COMPANY','FUll_NAME','COMPANY_SIZE','MEMBER_RATING'],
}  

file_table_mapping={
    'Unsubscribed':'UNSUB',

    'Didnt_open':'UNOPEN',
    
    'Bounced':'BOUNCE',
    
    'Clicked':'CLICKED',
    
    'Opened':'OPEN',
    
    'Sent_to':'SENT'
}

Download_Dir = {
    'MailChimp': '/Users/karim/Downloads/Mailchimp_downloads/'
}

def browser_init(workflow):
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory' : Download_Dir['MailChimp']}
    chrome_options.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome(executable_path='/Library/Application Support/Google/chromedriver', chrome_options=chrome_options)
    return browser



def browser_init(workflow):
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory' : Download_Dir['MailChimp']}
    chrome_options.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome(executable_path='/Library/Application Support/Google/chromedriver', chrome_options=chrome_options)
    return browser



def mailchimp_login(browser):
    #Go to webpage
    browser.get("https://mailchimp.com/")
    
    #Login Menu --> will ask for verification
    WebDriverWait(browser,10).until(EC.element_to_be_clickable((By.CLASS_NAME,'globalNav__actions__login'))).click() #login_button 

    #Find username/pass fields
    username = WebDriverWait(browser,4).until(EC.element_to_be_clickable((By.NAME, 'username')))
    password = WebDriverWait(browser,4).until(EC.element_to_be_clickable((By.NAME, 'password')))

    #input username and pass
    username.clear()
    username.send_keys('age_group')
    password.clear()
    password.send_keys('fourteen_gold')

    #press login button
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="login-form"]/fieldset/div[4]/button'))).click() #login_button 

    #email verification workflow
    try:
        WebDriverWait(browser, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="send-email-code-step1"]/button'))).click()
        WebDriverWait(browser, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-close-btn-container"]/button'))).click()
    except TimeoutException:
        pass
    verification_code = input("Code sent! Please check your email and use the box below to submit the verification code.")
    input_verification_code = browser.find_element(By.ID, 'email_code')
    input_verification_code.send_keys(verification_code)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="login-verify-fieldset"]/div[3]/div/button'))).click() #submit_verification 


def switch_to_fallback_iframe(browser):
    campaigns = browser.find_element(By.XPATH,"//iframe[@id='fallback']")
    browser.switch_to.frame(campaigns)

def convert_str_datetime(string):
    string =string.replace(',','').split(' ')
    string[-1] = string[-1].upper()
    string = ":".join(string)
    string= datetime.strptime(string, "%a:%b:%d:%Y:%I:%M:%p")
    return string

def locate_campaign_blocks(browser):
    browser.get("https://us7.admin.mailchimp.com/campaigns/")
    switch_to_fallback_iframe(browser)
    campaignManager = browser.find_elements(By.XPATH, '//*[@id="tabList"]/div/div/div/div[2]/div[2]/div[2]')
    campaignManager = browser.find_element(By.CLASS_NAME, 'c-campaignManager_list')
    monthly_lists= campaignManager.find_elements(By.CSS_SELECTOR, ".c-campaignManager_list_group.margin--lv4.margin-bottom--lv0.margin-right--lv0.margin-left--lv0")
    blocks = {}
    for i in monthly_lists:
        date = i.find_element(By.TAG_NAME,'h6').text
        print(f'date: {date}')
        block = i
        blocks[date]=block
    return monthly_lists

def get_monthly_campaigns(monthly_lists, index=0):
    campaigns = monthly_lists[index].find_elements(By.CSS_SELECTOR, ".c-campaignManager_slat.flex.position--relative.flex-between.flex-justify--start.padding--lv3.padding-left--lv1.padding-right--lv0")
    return campaigns

def processed_campaign_filter(campaigns):
    campaigns_to_process = []
    processed = pd.read_csv('../processed_campaigns.csv',header=0)["name"].tolist()
    with open('../processed_campaigns.csv', 'a', newline='') as f_object:
        writer_object = csv.writer(f_object)
        for c in campaigns:
            name = c.find_element(By.TAG_NAME, 'h4').find_element(By.TAG_NAME,'a').text
            report_link = c.find_element(By.XPATH,'./div[6]/table/tbody/tr/td[1]/div/div[2]/a').get_attribute('href')
            status = c.find_element(By.CSS_SELECTOR, '.c-campaignManager_slat_status.margin-bottom--lv2').find_element(By.CLASS_NAME,'badge').text
            if status == 'Sent':
                if name not in processed:
                    print(f'adding {name}')
                    campaigns_to_process.append(c)
                    single_campaign = [name,report_link]
                    writer_object.writerow(single_campaign) 
                else:
                    pass
            else:
                pass

    f_object.close()
    return campaigns_to_process

def get_campaigns_month(c):
    summary = {}
        
    name = c.find_element(By.TAG_NAME, 'h4').find_element(By.TAG_NAME,'a').text
    status = c.find_element(By.CSS_SELECTOR, '.c-campaignManager_slat_status.margin-bottom--lv2').find_element(By.CLASS_NAME,'badge').text
    report_link = c.find_element(By.XPATH,'./div[6]/table/tbody/tr/td[1]/div/div[2]/a').get_attribute('href')
    if status == 'Sent':
        print(f'Processing {name}')
        summary[name]={}
        summary[name]['status']=status
        summary[name]['link']=report_link
        df = pd.DataFrame.from_dict(summary, orient='index')
        return df
    else:
        print(f'Not sent yet: {name}, skipping')
    
    return summary
    

def get_activity_links(browser,df):
    for x in range(len(df)):
        browser.get(df.iloc[x,1])
        switch_to_fallback_iframe(browser)
        
        activity_blocks  = browser.find_element(By.XPATH,'//*[@id="dijit__FocusMixin_1"]').find_elements(By.TAG_NAME,'li')
        date = browser.find_element(By.XPATH,'//*[@id="report-wrapper"]/div/div[1]/div[2]/div[2]/ul/li[1]/span[2]').text
        df['date'] = convert_str_datetime(date)
        activity_links = []

        print(df.index[0])
        for y in activity_blocks:
            link = y.find_element(By.TAG_NAME,'a').get_attribute('href')
            section = y.find_element(By.TAG_NAME,'a').get_attribute('text')
            #print(section, link, date)
            activity_links.append([section, link])

        activity_df = pd.DataFrame(activity_links, columns=['section', 'link'])

    return activity_df

def download_activity_report(browser,activity_df, Download_Dir=Download_Dir['MailChimp']):
    idx = activity_df.shape[0]
    for ix in range(idx):
        section = activity_df.iloc[ix,0]
        link = activity_df.iloc[ix,1]
        browser.get(link)
        switch_to_fallback_iframe(browser)
        print(f'index: {ix}, section: {section}, link {link}')
        
        if ix == 0:
            browser.find_element(By.XPATH,'//*[@id="report-wrapper"]/div/div[2]/div[1]/div[1]/div[2]/span/a').click() #This Xpath for activity_link[0]
            sleep(2)
            tiny_file_rename(f'{section}.csv',Download_Dir)
        else:
            try:
                browser.find_element(By.CSS_SELECTOR,'.button.hide-phone').click() #This Xpath for activity_link[1:], needs to be put in a try except block as some of the link won't have exports available (e.g: if no one unsubscribed)
                sleep(2)
                tiny_file_rename(f'{section}.csv',Download_Dir)
            except Exception as e:
                print(e)


def tiny_file_rename(newname, folder_of_download, time_to_wait=60):
    newname = newname.replace(' ','_').replace("'","")
    filename = max([f for f in os.listdir(folder_of_download)], key=lambda xa :   os.path.getctime(os.path.join(folder_of_download,xa)))
    os.rename(os.path.join(folder_of_download, filename), os.path.join(folder_of_download, newname))


def reports_csv_to_dfs(download_dir):
    df_dict = {}
    for i in os.listdir(download_dir):
        if i.endswith('.csv'):
            df_name = i.replace('.csv','')
            report = i.replace('.csv','')
            print(df_name)
            print(type(df_name))
            file_dir = f"{Download_Dir['MailChimp']+i}"
            print(file_dir)
            exec(f"{report} = pd.read_csv('{file_dir}', header=0, index_col=False)")
            exec(f"df_dict['{df_name}']={report}")
            os.remove(download_dir + i)
    return df_dict


def process_df_columns(df_dict,summary_df,db_cols):
    for k,v  in df_dict.items(): 
        try:
            print('..1')
            print(k)
            print(v.columns)
            df_dict[k].rename(columns=col_renames[k], errors='ignore', inplace=True)
            print(f'db_cols[k]: {db_cols[k]}')
            df_dict[k] = df_dict[k][db_cols[k]]
            print(f'{k}.cols: \n{df_dict[k].columns}')
            # v.drop(columns = ['UID','MAILCHIMP_IMPORT','CLICKUP_IMPORT','Index'], axis=1,inplace=True)
            df_dict[k]['CAMPAIGN'] = summary_df.index[0]
            print('..2')
            print(summary_df.iloc[0,2])
            df_dict[k]['DATE'] = summary_df.iloc[0,2]
            print('..3')
            print(v.columns)
            print(v.head())

        except Exception as e:
            print('..4')
            print(e)

def send_to_database(df_dict,engine):
    for k,v  in df_dict.items(): 
        try:
            df_dict[k].to_sql(file_table_mapping[k], con=engine,index=False,  if_exists='append')
        except Exception as e:
            print(e)

In [64]:
engine = create_engine('sqlite:///../mailchimp.db', echo=True)

# Automation Flow

In [65]:
#from libs import *

In [66]:
#LOGIN 
browser = browser_init('MailChimp')
mailchimp_login(browser)

/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:126: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:126: DeprecationWarning: use options instead of chrome_options


In [67]:
#blocks for each month //*[@id="dijit__WidgetsInTemplateMixin_2"]/h6
monthly_lists = locate_campaign_blocks(browser)
len(monthly_lists)

date: February, 2022 (1)
date: January, 2022 (14)
date: December, 2021 (4)
date: July, 2021 (2)
date: May, 2021 (1)
date: April, 2021 (2)
date: December, 2020


7

In [68]:
#break down the monthly blocks into the specific <li> tag elements
master_list = []
for x in range(len(monthly_lists)):
    campaign = get_monthly_campaigns(monthly_lists, index=x)
    #print(type(campaign))
    #print(len(campaign))
    for i in range(len(campaign)):
        #print(type(campaign[i]))
        master_list.append(campaign[i])

In [69]:
## to_process_df --> list of campaigns' webelemments that will be processed, verifies by verifying against a csv of campaigns already processed
campaigns_to_process_webelements = processed_campaign_filter(master_list)

adding 006 - 05- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new contacts
adding 006 - 06- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new contacts
adding 005 - 04 - Pro-Consult - Real Estate Campaign - New contacts from 25th Jan (copy 01)
adding 005 - 03 - Pro-Consult - Real Estate Campaign - New contacts from 25th Jan
adding 007 - 02- Pro-Consult Legal - Looking for trusted Legal Counsel?
adding 006 - 04 - Pro-Consult Legal - Your Contacts: More Valuable Than You Think
adding 007 - Pro-Consult Legal - Looking for trusted Legal Counsel?


In [70]:
# create a list of dataframes, each containing 1 row with name, link and date
monthly_campaign_dfs = []
for i in range(len(campaigns_to_process_webelements)):
    df = get_campaigns_month(campaigns_to_process_webelements[i])
    monthly_campaign_dfs.append(df)

Processing 006 - 05- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new contacts
Processing 006 - 06- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new contacts
Processing 005 - 04 - Pro-Consult - Real Estate Campaign - New contacts from 25th Jan (copy 01)
Processing 005 - 03 - Pro-Consult - Real Estate Campaign - New contacts from 25th Jan
Processing 007 - 02- Pro-Consult Legal - Looking for trusted Legal Counsel?
Processing 006 - 04 - Pro-Consult Legal - Your Contacts: More Valuable Than You Think
Processing 007 - Pro-Consult Legal - Looking for trusted Legal Counsel?


In [71]:
#monthly_campaign_dfs[0]
#monthly_campaign_dfs[0].dtypes


In [72]:
activties_dfs=[]
for x in range(len(monthly_campaign_dfs)):
    activity_links_df = get_activity_links(browser,monthly_campaign_dfs[x])
    activties_dfs.append(activity_links_df)

print(len(activties_dfs))


006 - 05- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new contacts
006 - 06- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new contacts
005 - 04 - Pro-Consult - Real Estate Campaign - New contacts from 25th Jan (copy 01)
005 - 03 - Pro-Consult - Real Estate Campaign - New contacts from 25th Jan
007 - 02- Pro-Consult Legal - Looking for trusted Legal Counsel?
006 - 04 - Pro-Consult Legal - Your Contacts: More Valuable Than You Think
007 - Pro-Consult Legal - Looking for trusted Legal Counsel?
7


In [73]:
#monthly_campaign_dfs[0]
#activties_dfs[0]
#df_dict.keys()
#df_dict['Opened'].columns
#activties_dfs[0]

In [74]:
for idx in range(len(activties_dfs)):
#for idx in [0]:
    download_activity_report(browser,activties_dfs[idx], Download_Dir=Download_Dir['MailChimp'])
    df_dict = reports_csv_to_dfs(Download_Dir['MailChimp'])
    process_df_columns(df_dict,monthly_campaign_dfs[idx],db_cols)
    send_to_database(df_dict,engine)

index: 0, section: Sent to, link https://us7.admin.mailchimp.com/reports/activity/sent?id=13644585
index: 1, section: Opened, link https://us7.admin.mailchimp.com/reports/activity/opened?id=13644585
index: 2, section: Didn't open, link https://us7.admin.mailchimp.com/reports/activity/not-opened?id=13644585
index: 3, section: Clicked, link https://us7.admin.mailchimp.com/reports/activity/clicked?id=13644585
Message: no such element: Unable to locate element: {"method":"css selector","selector":".button.hide-phone"}
  (Session info: chrome=98.0.4758.80)
Stacktrace:
0   chromedriver                        0x0000000102d963c9 chromedriver + 5018569
1   chromedriver                        0x0000000102d21333 chromedriver + 4539187
2   chromedriver                        0x00000001028f6a88 chromedriver + 170632
3   chromedriver                        0x000000010292b332 chromedriver + 385842
4   chromedriver                        0x000000010292b4f1 chromedriver + 386289
5   chromedriver       

/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

2022-02-06 14:50:37,835 INFO sqlalchemy.engine.Engine INSERT INTO "SENT" ("EMAIL", "FIRST_NAME", "LAST_NAME", "ADDRESS", "PHONE", "COMPANY", "FUll_NAME", "COMPANY_SIZE", "MEMBER_RATING", "CAMPAIGN", "DATE") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-02-06 14:50:37,835 INFO sqlalchemy.engine.Engine [generated in 0.02014s] (('ali@espace.ae', 'ALI', 'AMIRI', None, '1000000000', 'ESPACE REAL ESTATE BROKER', 'ALI AHMED AMIRI', 88, 2, '006 - 05- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new contacts', '2022-01-31 18:00:00.000000'), ('anit@phrealestate.ae', 'ANIT', 'MADHANI', None, '44087000', 'PH REAL ESTATE BROKERS', 'ANIT MADHANI', 16, 2, '006 - 05- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new contacts', '2022-01-31 18:00:00.000000'), ('shekar@nwdubai.com', 'CHANDRA', 'PATUKARI', None, '44309901', 'SHELTERS REAL ESTATE BROKER', 'CHANDRA SHEKAR PATUKARI', 4, 2, '006 - 05- Pro-Consult Legal - Your Contacts: More Valuable Than You Think- new c

/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

index: 0, section: Sent to, link https://us7.admin.mailchimp.com/reports/activity/sent?id=13628593
index: 1, section: Opened, link https://us7.admin.mailchimp.com/reports/activity/opened?id=13628593
index: 2, section: Didn't open, link https://us7.admin.mailchimp.com/reports/activity/not-opened?id=13628593
index: 3, section: Clicked, link https://us7.admin.mailchimp.com/reports/activity/clicked?id=13628593
index: 4, section: Bounced, link https://us7.admin.mailchimp.com/reports/activity/bounced?id=13628593
index: 5, section: Unsubscribed, link https://us7.admin.mailchimp.com/reports/activity/unsubscribed?id=13628593
index: 6, section: Complained, link https://us7.admin.mailchimp.com/reports/activity/abuse?id=13628593
Message: no such element: Unable to locate element: {"method":"css selector","selector":".button.hide-phone"}
  (Session info: chrome=98.0.4758.80)
Stacktrace:
0   chromedriver                        0x0000000102d963c9 chromedriver + 5018569
1   chromedriver               

...

/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

index: 0, section: Sent to, link https://us7.admin.mailchimp.com/reports/activity/sent?id=13563352
index: 1, section: Opened, link https://us7.admin.mailchimp.com/reports/activity/opened?id=13563352
index: 2, section: Didn't open, link https://us7.admin.mailchimp.com/reports/activity/not-opened?id=13563352
index: 3, section: Clicked, link https://us7.admin.mailchimp.com/reports/activity/clicked?id=13563352
Message: no such element: Unable to locate element: {"method":"css selector","selector":".button.hide-phone"}
  (Session info: chrome=98.0.4758.80)
Stacktrace:
0   chromedriver                        0x0000000102d963c9 chromedriver + 5018569
1   chromedriver                        0x0000000102d21333 chromedriver + 4539187
2   chromedriver                        0x00000001028f6a88 chromedriver + 170632
3   chromedriver                        0x000000010292b332 chromedriver + 385842
4   chromedriver                        0x000000010292b4f1 chromedriver + 386289
5   chromedriver       

/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

index: 0, section: Sent to, link https://us7.admin.mailchimp.com/reports/activity/sent?id=5503776
index: 1, section: Opened, link https://us7.admin.mailchimp.com/reports/activity/opened?id=5503776
index: 2, section: Didn't open, link https://us7.admin.mailchimp.com/reports/activity/not-opened?id=5503776
index: 3, section: Clicked, link https://us7.admin.mailchimp.com/reports/activity/clicked?id=5503776
index: 4, section: Bounced, link https://us7.admin.mailchimp.com/reports/activity/bounced?id=5503776
index: 5, section: Unsubscribed, link https://us7.admin.mailchimp.com/reports/activity/unsubscribed?id=5503776
index: 6, section: Complained, link https://us7.admin.mailchimp.com/reports/activity/abuse?id=5503776
Message: no such element: Unable to locate element: {"method":"css selector","selector":".button.hide-phone"}
  (Session info: chrome=98.0.4758.80)
Stacktrace:
0   chromedriver                        0x0000000102d963c9 chromedriver + 5018569
1   chromedriver                      

/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/karim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 